<div class="alert alert-block alert-info"> <b>NOTE</b> 

<div>
    This is a duplicate of notebook <b>03_02_no_temporal_data_wrangling.ipynb </b>
    in this notebook I want to split the data into training/validation//testing sets.
    I'll use an 80 (training/validation)//20 (testing) split
</div>
</div>



In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from auxiliary_functions import write_joblib
from data_wrangling import load_all_pickles, remove_small_dfs, remove_undesired_columns, remove_remaining_data, length_total, combined_items

In [2]:
# Load dictionaries 
folder_path = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/02_Clotting_Labelling' 
path_to_save1 = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/03_01_Train_Val_Test'
loaded_pickles = load_all_pickles(folder_path)

no_clotting_dict = {}
clotting_dict = {}

# Iterate through the original dictionary and sort based on the key
for key, value in loaded_pickles.items():
    if "no_clotting" in key:
        no_clotting_dict[key] = value
    elif "clotting" in key:
        clotting_dict[key] = value


Now I want to run the quality control. I need to iterate through each .pkl file. Each file is composed of multiple time series; I want to discard those whose len(time series) < 70. I would like to print the len of each dictionary after running this test    

In [3]:
columns=['Date__Heure', 'trt', 'Patient_weight__Kg_' , 'Set', 'Condition_1', 'Condition_2', 'Delta_P_ref', 'TMP_ref', 'Clotting_1', 'group']
min_length = 70
shift = 15  # number of points we want to conserve once we identify a blocking event 
# Filter for removing small Data Frames (length)
remove_small_dfs(no_clotting_dict, min_length)
remove_small_dfs(clotting_dict, min_length)

#Filter for removing undesired columns
remove_undesired_columns(no_clotting_dict,columns)
remove_undesired_columns(clotting_dict,columns)

#Filter for cutting, in the case of blocking data, the elements after the blocking event
remove_remaining_data(clotting_dict)
#remove_undesired_data_custom(clotting_dict, shift)

<div class="alert alert-block alert-info"> <b>Note regarding remove_undesired_data_custom() function.</b> 

With the results obtained so far, it seems that this function is obsolete since the model performs better when we add as many blocking points as possible. This is easily done with the function remove_undesired_columns()
</div>

Lets see how many data frames from each class do we have

In [4]:
length_total(clotting_dict)

completo1007_(edit)_clotting.pkl 18
completo600_(edit)_clotting.pkl 15
completo400_(edit)_clotting.pkl 19
completo_800_output_file_clotting.pkl 25
completo200_(edit)_clotting.pkl 14


In [5]:
length_total(no_clotting_dict)

completo400_(edit)_no_clotting.pkl 121
completo600_(edit)_no_clotting.pkl 155
completo1007_(edit)_no_clotting.pkl 157
completo200_(edit)_no_clotting.pkl 141
completo_800_output_file_no_clotting.pkl 131


1. I want to merge, randomly, all the timeseries of each dictionary into a global one. At the end of this process I'll have blocking_dict and no_blocking_dict
2. I want to split each global dictionary into two subsets 90/10 so I can (train,validate)/test. Therefore, I'll have
   blocking_dict_tval
   blocking_dict_test
   no_blocking_dict_tval
   no_blocking_dict_test 

In [6]:
random.seed(42)

global_dict_block = {**combined_items(clotting_dict)}
items_block = list(global_dict_block.items())
random.shuffle(items_block)
global_dict_block = dict(items_block)

global_dict_no_block = {**combined_items(no_clotting_dict)}
items_no_block = list(global_dict_no_block.items())
random.shuffle(items_no_block)
global_dict_no_block = dict(items_no_block)

In [7]:
print(len(global_dict_block))
print(len(global_dict_no_block))

91
705


In [10]:
total_keys_block = list(global_dict_block.keys())
split_index_block = int(len(global_dict_block) * 0.8)

keys_train_val_percent_block = random.sample(total_keys_block, split_index_block)
dict_train_val_block = {key: global_dict_block[key] for key in keys_train_val_percent_block}
keys_test_percent_block = list(set(total_keys_block) - set(keys_train_val_percent_block))
dict_test_block = {key: global_dict_block[key] for key in keys_test_percent_block}

In [11]:
print(len(dict_train_val_block))

print(len(dict_test_block))

72
19


In [12]:
total_keys_no_block = list(global_dict_no_block.keys())
split_index_no_block = int(len(global_dict_no_block) * 0.8)

keys_train_val_percent_no_block = random.sample(total_keys_no_block, split_index_no_block)
dict_train_val_no_block = {key: global_dict_no_block[key] for key in keys_train_val_percent_no_block}
keys_test_percent_no_block = list(set(total_keys_no_block) - set(keys_train_val_percent_no_block))
dict_test_no_block = {key: global_dict_no_block[key] for key in keys_test_percent_no_block}

In [14]:
print(len(dict_train_val_no_block))
print(len(dict_test_no_block))

564
141


In [15]:
# Save the dictionaries
'''name_save = f'{path_to_save1}/blocking_80.pkl' 
with open(name_save, 'wb') as file:
    pickle.dump(dict_90_block, file)


name_save = f'{path_to_save1}/blocking_20.pkl' 
with open(name_save, 'wb') as file:
    pickle.dump(dict_10_block, file)

name_save = f'{path_to_save1}/no_blocking_80.pkl' 
with open(name_save, 'wb') as file:
    pickle.dump(dict_90_no_block, file)
    
name_save = f'{path_to_save1}/no_blocking_20.pkl' 
with open(name_save, 'wb') as file:
    pickle.dump(dict_10_no_block, file)'''


write_joblib(dict_train_val_block, path_to_save1, 'blocking_train_val.pkl')
write_joblib(dict_train_val_block, path_to_save1, 'blocking_test.pkl')
write_joblib(dict_train_val_block, path_to_save1, 'non_blocking_train_val.pkl')
write_joblib(dict_train_val_block, path_to_save1, 'non_blocking_test.pkl')
